## Google Drive Authentication

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('drive', 'v3', credentials=creds)
    service_sheets = build('sheets', 'v4', credentials=creds)
    
    print("** Authentication Complete! **")

except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')


** Authentication Complete! **


## Google Drive API helper functions

In [2]:
import io
from googleapiclient.http import MediaIoBaseDownload
from os import listdir

def getFolderId(service, folderName: str):
    query = "name contains '%s' and mimeType = '%s'" % (folderName, 'application/vnd.google-apps.folder')

    fid = None

    if folderName.startswith('+'):
        return (folderName[1:])

    result = service.files().list(q=query,
                                  pageSize=10, pageToken='',
                                  fields="nextPageToken,files(parents,id,name,mimeType)").execute()
  
    if len(result['files']) == 0:
        print("Folder NOT found")
    else:
        folder = result.get('files')[0]
        fid = folder['id']

    return (fid)

    
def downloadFolder(service, fileId, destinationFolder):
    if not os.path.isdir(destinationFolder):
        os.mkdir(path=destinationFolder)

    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(fileId),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])

    for item in items:
        itemName = item['name']
        itemId = item['id']
        itemType = item['mimeType']
        filePath = destinationFolder + "/" + itemName

        if itemType == 'application/vnd.google-apps.folder':
            downloadFolder(service, itemId, filePath) # Recursive call
            print("Downloaded folder: {0}".format(itemName))
        elif not itemType.startswith('application/'):
            downloadFile(service, itemId, filePath)
        else:
            print("Unsupported file: {0}".format(itemName))


def downloadFile(service, fileId, filePath):
    # Note: The parent folders in filePath must exist
    request = service.files().get_media(fileId=fileId)
    fh = io.FileIO(filePath, mode='wb')
    
    try:
        downloader = MediaIoBaseDownload(fh, request, chunksize=1024*1024)

        done = False
        while done is False:
            status, done = downloader.next_chunk(num_retries = 2)
    finally:
        fh.close()
        
def deleteFilesInFolder(folder_id):
    results = service.files().list(
        pageSize=300,
        q="parents in '{0}'".format(folder_id),
        fields="files(id, name, mimeType)"
        ).execute()

    items = results.get('files', [])
    for item in items:
        itemId = item['id']
        service.files().delete(fileId=itemId).execute()
        
def uploadFolder(service, folder_id, src_folder):        
    for file in listdir(src_folder):
        print('Uploading: ' + file)
        file_metadata = {'name': file, 'parents': [folder_id]}
        media = MediaFileUpload(os.path.join(src_folder, file), mimetype='image/png')
        file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()

## Download Trait Files!

In [4]:
import shutil

traits_base_filepath = 'Traits'

# Delete previously downloaded trait files
if os.path.isdir(traits_base_filepath):
    shutil.rmtree(traits_base_filepath)

traits_folder_id = getFolderId(service, traits_base_filepath)

downloadFolder(service, traits_folder_id, traits_base_filepath)

print("\n** Download Complete! **")

Downloaded folder: ears
Downloaded folder: patch
Downloaded folder: satchels
Downloaded folder: quarter zip
Downloaded folder: upper + lower
Downloaded folder: short pants
Downloaded folder: hats + face
Downloaded folder: pillow
Downloaded folder: headphones
Downloaded folder: spilled carton
Downloaded folder: bucket hats
Downloaded folder: Flag Patches
Downloaded folder: Patches
Downloaded folder: beanies
Downloaded folder: Patches
Downloaded folder: baseball cap
Downloaded folder: hats
Downloaded folder: walkman
Downloaded folder: halo
Downloaded folder: dots
Downloaded folder: healthbar
Downloaded folder: external
Downloaded folder: chinese mask
Downloaded folder: syringe
Downloaded folder: sticky notes
Downloaded folder: glasses
Downloaded folder: fb
Downloaded folder: surgical masks
Downloaded folder: face
Downloaded folder: satchels
Downloaded folder: Meta Tees
Downloaded folder: short sleeves
Downloaded folder: fat pants
Downloaded folder: NB Sneakers
Downloaded folder: AirForce

## Print all folders, files (BFS)

In [101]:
folder_queue = [traits_base_filepath]

#Generate rarity definitions
while folder_queue:
    curr_folder = folder_queue.pop(0)
    
    files = os.listdir(curr_folder)
    files = filter(lambda file: not file.startswith('.'), files)
    
    for file in files:
        print(file)
        if not file.lower().endswith('.png'):
            folder_queue.append(os.path.join(curr_folder, file))
        

short pants
hoodies
quarter zip
face
short sleeves
hats
body
upper + lower
ears
fat pants
slim pants
dress shirt
undergarment upper
hats + face
external
shoes
undergarment lower
shorts_black.PNG
shorts_grey.PNG
shorts_red.PNG
shorts_brown.PNG
shorts_blue.PNG
shorts_purple.PNG
shorts_green.PNG
Hoodies Plain
Hoodies LP
quarterzip_red.PNG
quarterzip_pink.PNG
quarterzip_purple.PNG
quarterzip_green.PNG
quarterzip_grey.PNG
surgical masks
fb
sticky notes
glasses
syringe
Meta Tees
baseball cap
bucket hats
beanies
headphones
pillow_white.PNG
spilled carton
eaten
eaten catalien
crying
grin
neutral
angry
laugh
charlesharness_red.PNG
earrings
earbleed_white.PNG
jeans_lightblue.PNG
jeans_darkblue.PNG
Sweatpants Plain
Sweatpants LP
dressshirt_pink.PNG
dressshirt_grey.PNG
wifebeater_pink.png
wifebeater_white.png
Space_Helmet.PNG
walkman_green.PNG
dots
halo_yellow.PNG
healthbar
NB Sneakers
AirForce1
underwear_grey.png
underwear_white.png
Hoodie_Green.PNG
Hoodie_Vanilla.PNG
Hoodie_Grey.PNG
Hoodie_White

## Some more helper functions

In [14]:
from collections import deque
import random

# folder: subtrait
# traits_already_picked: keep track
# returns: chosen image(s), new set of already picked traits
def pick_trait_images(folder, traits_already_picked):
    
    png_files, folder_files = get_compatible_files(folder, traits_already_picked)
    
    chosen_image = rarity_chooser(png_files)
    chosen_folder = rarity_chooser(folder_files)
    
    traits_picked = []
    if chosen_image:
        traits_picked = get_traits_for_image(os.path.join(folder, chosen_image))
    
    # If only PNGs in folder
    if len(folder_files) == 0:
        return [os.path.join(folder, chosen_image)], traits_already_picked + traits_picked
    
    # If only folders in folder
    elif len(png_files) == 0:
        return pick_trait_images(os.path.join(folder, chosen_folder), traits_already_picked)
    
    # Both PNGs and folders in folder
    else:
        chosen_image_path = os.path.join(folder, chosen_image)
        picked_image, picked_traits = pick_trait_images(os.path.join(folder, chosen_folder), traits_already_picked + traits_picked)
        return [chosen_image_path] + picked_image, traits_already_picked + traits_picked + picked_traits
                                                        
# Randomly choosing for now
# elems: list of PNGs/folders
def rarity_chooser(elems):
    if elems:
        return random.choice(elems)
    return None

# folder:
# traits_already_picked:
# returns: compatible files within folder that are compatible with traits_already_picked
def get_compatible_files(folder, traits_already_picked):
    files = os.listdir(folder)
    
    # Drop pesky ./DSStore files
    files = filter(lambda file: not file.startswith('.'), files)
    
    png_files = []
    folder_files = []
    for file in files:
        if file.lower().endswith('.png'):
            trait_name, color, _ = parse_png_filename(file)            
        
            if is_possible_choice(trait_name, exclusions_dict, traits_already_picked) & \
                is_possible_choice(color, exclusions_dict, traits_already_picked):
                png_files.append(file)
        else:
            if is_possible_choice(file, exclusions_dict, traits_already_picked):
                folder_files.append(file)
   

    return png_files, folder_files

def is_possible_choice(entry, exclusions_dict, traits_already_picked):
    return (entry not in exclusions_dict or not(set(exclusions_dict[entry]) & set(traits_already_picked)))

def get_traits_for_image(chosen_image):
    traits = chosen_image.split('/')
    trait_name, color, _ = parse_png_filename(traits[-1])
    traits[-1] = trait_name
    traits.append(color)
    return traits
    
def parse_png_filename(png_filename):
    parsed_attributes = png_filename.split('_')
    if len(parsed_attributes) == 1:
        return parsed_attributes[0], "", ""
    elif len(parsed_attributes) == 2:
        return parsed_attributes[0], parsed_attributes[1], ""
    elif len(parsed_attributes) == 3:
        return parsed_attributes[0], parsed_attributes[1], parsed_attributes[2]

## Generate Babies!

In [10]:
# Run this on update of either 1. layering order and 2. exclusions sheet

sheet = service_sheets.spreadsheets()

LAYERING_ORDER_SPREADSHEET_ID = '1aHC5g3mPSJGFAPF7UiQXDnV9BBUJCwnx8QHkMMip6uI'
LAYERING_ORDER_RANGE = 'A1:A'

EXCLUSIONS_SPREADSHEET_ID = '1S3Gbg24gwCmn_2AwThAIlRN9F0PEbA2v1Kn3QkaTjeY'
EXCLUSIONS_RANGE = 'A1:B'

result = sheet.values().get(spreadsheetId=LAYERING_ORDER_SPREADSHEET_ID, range=LAYERING_ORDER_RANGE).execute()
values = result.get('values', [])

ordered_traits = [item for sublist in values for item in sublist]

print("** Traits Ordering **\n")
for trait in ordered_traits:
    print(trait + '\n')
    
result = sheet.values().get(spreadsheetId=EXCLUSIONS_SPREADSHEET_ID, range=EXCLUSIONS_RANGE).execute()
values = result.get('values', [])

exclusions_dict = {}
for val in values:
    exclusions = val[1].split(',')
    for e in exclusions:
        if e not in exclusions_dict:
                exclusions_dict[e] = []
        exclusions_dict[e].append(val[0])

print("** Traits Exclusions **\n")
for key, val in exclusions_dict.items():
    print(key + ": " + str(val) + "\n")
        
images_count = 4

num_traits_excluding_body = 5

babies_base_filepath = 'Babies'

** Traits Ordering **

body

upper + lower

undergarment lower

short pants

slim pants

undergarment upper

shoes

fat pants

short sleeves

dress shirt

quarter zip

hoodies

face

hats

hats + face

ears

external

** Traits Exclusions **

surgical masks: ['eaten catalien', 'eaten']

baseball cap: ['eaten catalien', 'eaten']

spilled carton: ['eaten catalien', 'eaten']

beanies: ['eaten catalien', 'eaten']

hats + face: ['eaten catalien', 'eaten', 'bucket hats', 'spilled carton', 'beanies']

headphones: ['eaten catalien', 'eaten']

face: ['eaten catalien', 'eaten']

undergarment lower: ['charlesharness_red']

undergarment upper: ['charlesharness_red']

 short pants: ['charlesharness_red']

slim pants: ['charlesharness_red', 'short pants']

fat pants: ['charlesharness_red', 'short pants', 'slim pants']

short sleeves: ['charlesharness_red', 'satchel']

dress shirt: ['charlesharness_red', 'satchel', 'short sleeves']

hoodies: ['charlesharness_red', 'quarter zip', 'satchel']

surgicalm

In [52]:
from PIL import Image, ImageDraw, ImageChops

# top level
def generate_baby(ordered_traits, traits_base_filepath, babies_base_filepath):
    
    # BFS for trait images selection
    picked_trait_images = []
    picked_traits = []
    base_traits = ordered_traits
    
    # 1. choose the body trait
    body_trait_folder = base_traits.pop(0)
    curr_trait_dir = os.path.join(traits_base_filepath, body_trait_folder)
    picked_trait_image, picked_traits = pick_trait_images(curr_trait_dir, picked_traits)
    picked_trait_images.extend(picked_trait_image)
    
    # 2. Choose 0-2 face traits
    face_traits = ['face', 'hats', 'hats + face', 'ears']
    num_face_traits = random.randint(0, 2)
    chosen_face_traits = [face_traits[i] for i in sorted(random.sample(range(len(face_traits)), num_face_traits))]
    
    
    # 3. Randomly pick `num_traits_excluding_body` traits
    base_traits = list(set(base_traits) - set(face_traits))
    num_traits_excluding_body_face = num_traits_excluding_body - num_face_traits
    chosen_base_traits = [base_traits[i] for i in sorted(random.sample(range(len(base_traits)), num_traits_excluding_body_face))]
    
    unsorted_chosen_traits = chosen_base_traits + chosen_face_traits 
    final_chosen_traits = [trait for x in ordered_traits for trait in unsorted_chosen_traits if trait == x]

    print("Chosen traits: " + str(final_chosen_traits))
    
    while final_chosen_traits:
        curr = final_chosen_traits.pop(0)
        if curr in exclusions_dict and (set(exclusions_dict[curr]) & set(picked_traits)):
            continue
            
        curr_trait_dir = os.path.join(traits_base_filepath, curr)
        
        # pick based on exclusions
        picked_trait_image, picked_traits = pick_trait_images(curr_trait_dir, picked_traits)
        picked_trait_images.extend(picked_trait_image)
        
    # Layer the images 
    x, y = Image.open(picked_trait_images[0]).size
    final_baby_image = Image.new('RGB', (x, y), (228, 150, 150))
    
    for trait_image in picked_trait_images:
        chosen_image = Image.open(trait_image)
        chosen_image = ImageChops.offset(chosen_image, 90, 0)
        final_baby_image.paste(chosen_image, (0, 0), chosen_image)
    
    # Crop to increase baby appearance
    final_baby_image = final_baby_image.crop((0, 140, final_baby_image.width - 140, final_baby_image.height))
    
    # Resize to 1080 * 1080 (Instagram Image dimensions)
    final_baby_image = final_baby_image.resize((1080, 1080), Image.LANCZOS) 
    
    return final_baby_image


## Generate babies!

# Delete previously created baby files
if os.path.isdir(babies_base_filepath):
    shutil.rmtree(babies_base_filepath)

os.mkdir(babies_base_filepath)

for i in range(images_count):
    final_baby_image = generate_baby(ordered_traits.copy(), traits_base_filepath, babies_base_filepath)
    
    # Write the image to file
    final_baby_image_file = '{:d}_lonely_baby.PNG'.format(i)
    final_baby_image.save(os.path.join(babies_base_filepath, final_baby_image_file))
    
    print("Completed " + final_baby_image_file)

print("\n**Image generation complete! **")

Chosen traits: ['slim pants', 'fat pants', 'short sleeves', 'ears', 'external']
Completed 0_lonely_baby.PNG
Chosen traits: ['undergarment lower', 'short pants', 'slim pants', 'hats', 'ears']
Completed 1_lonely_baby.PNG
Chosen traits: ['upper + lower', 'undergarment lower', 'short pants', 'fat pants', 'external']
Completed 2_lonely_baby.PNG
Chosen traits: ['undergarment upper', 'shoes', 'fat pants', 'hats', 'ears']
Completed 3_lonely_baby.PNG

**Image generation complete! **


## Upload Babies to Google Drive

In [37]:
babies_folder_id = getFolderId(service, babies_base_filepath)
deleteFilesInFolder(babies_folder_id)
uploadFolder(service, babies_folder_id, os.path.join(babies_base_filepath))

Uploading: 8_lonely_baby.PNG
Uploading: 7_lonely_baby.PNG
Uploading: 0_lonely_baby.PNG
Uploading: 1_lonely_baby.PNG
Uploading: 9_lonely_baby.PNG
Uploading: 6_lonely_baby.PNG
Uploading: 3_lonely_baby.PNG
Uploading: 4_lonely_baby.PNG
Uploading: 5_lonely_baby.PNG
Uploading: 2_lonely_baby.PNG
